# Домашнее задание к занятию
## "Функции потерь и оптимизация"

- Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/
- Реализовать самостоятельно логистическую регрессию
- Обучить ее методом градиентного спуска
- Методом nesterov momentum
- Методом rmsprop

### Дополнительное задание *
В качестве dataset’а взять Iris, оставив 2 класса:
Iris Versicolor
Iris Virginica

In [1]:
# Загружаем необходимые библиотеки
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import math

In [2]:
# Загружаем датасет Iris
iris = datasets.load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = iris.target
df_iris['name'] = df_iris.target.apply(lambda x : iris.target_names[x])
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [3]:
# Получаем основную информацию о данных в откорректированном датасете
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int32  
 5   name               150 non-null    object 
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [4]:
df_iris['name'].value_counts()

virginica     50
versicolor    50
setosa        50
Name: name, dtype: int64

Пропущенные значения отстствуют.

In [5]:
# Для дальнейшей работе оставляем два класса: Iris Versicolor и Iris Virginica
df_iris = df_iris.loc[df_iris['name'].isin(['virginica', 'versicolor'])]
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,name
50,7.0,3.2,4.7,1.4,1,versicolor
51,6.4,3.2,4.5,1.5,1,versicolor
52,6.9,3.1,4.9,1.5,1,versicolor
53,5.5,2.3,4.0,1.3,1,versicolor
54,6.5,2.8,4.6,1.5,1,versicolor


In [6]:
# Разделим целевую переменную и данные, по которым будет осуществляться предсказание
le = LabelEncoder()
le.fit(df_iris['target'])
df_iris['target'] = le.transform(df_iris['target'])
y = pd.Series(df_iris['target'])
df_iris.pop('name')
df_iris.pop('target')
X = df_iris

In [7]:
# Разделим данные на обучающие и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# Создадим модель логистической регрессии
model_1 = LogisticRegression()

In [9]:
# Обучаем на тренировочной части датасета и строим предсказание
model_1.fit(X_train, y_train)
predictions = model_1.predict(X_test)

In [10]:
# Выведем качество полученной модели на тестовой выборке
model_1.score(X_test,y_test)

0.9333333333333333

In [11]:
#  Функция по вероятности события предсказывает класс 0 или 1 по указанному пороговому значению вероятности
def prediction(pred, item):
        if pred >= item:
            return 1
        else:
            return 0

In [12]:
# Функция прогнозирует значения целевой переменной
def my_prediction(params, X):
    return 1 / (1 + np.e ** ((-1) * (params[0] + params[1] * X['sepal length (cm)'] 
    + params[2] * X['sepal width (cm)'] + params[3] * X['petal length (cm)'] 
    + params[4] * X['petal width (cm)']))) 

### Метод градиентного спуска

In [13]:
# Задаём исходные данные
lr = 0.1
n_epochs = 50
params = []
params = np.random.normal(size=(5,))

In [14]:
# Осуществляем градиентный спуск для нашей модели
for epoch in range(n_epochs):
    predictions_demo = my_prediction(params, X_train)
    params[0] -= lr * np.sum(predictions_demo - y_train) / len(predictions_demo)
    number = 1
    for i in X_train:
        params[number] -= lr * np.sum((predictions_demo - y_train) * X_train[i]) / len(predictions_demo)
        number += 1
    predictions = my_prediction(params, X_train).apply(prediction, item=0.5)

In [15]:
# Определим долю верных ответов на тренировочной выборке
pred = my_prediction(params, X_train)
new_pred = pred.apply(prediction, item=0.5)
1 - np.sum(abs(new_pred - y_train)) / len(y_train)

0.9142857142857143

In [16]:
# Определим долю верных ответов на тестовой выборке
pred = my_prediction(params, X_test)
new_pred = pred.apply(prediction, item=0.5)
1 - np.sum(abs(new_pred - y_test)) / len(y_test)

0.8666666666666667

### Nesterov momentum

In [17]:
# Задаём исходные данные
lr = 0.1
n_epochs = 50
params = []
params = np.random.normal(size=(5,))
gamma = 0.9
u_0 = new_u_0 = 0
u = new_u = [0, 0, 0, 0, 0]

In [18]:
# Nesterov momentum для нашей модели
for epoch in range(n_epochs):
    predictions_demo = my_prediction(params, X_train)
    new_u_0 = u_0 * gamma + (1 - gamma) * lr * np.sum(predictions_demo - y_train) / len(predictions_demo)
    number = 0
    for i in X_train:
        new_u[number] = u[number] * gamma + (1 - gamma) * lr * np.sum((predictions_demo - y_train) * X_train[i])
        number += 1
    for i in range(len(params)):
        if i == 0:
            params[i] -= new_u_0
        else:
            params[i] -= new_u[i-1]
    u_0 = new_u_0
    u = new_u
    predictions = my_prediction(params, X_train).apply(prediction, item=0.5)

In [19]:
# Определим долю верных ответов на тренировочной выборке
pred = my_prediction(params, X_train)
new_pred = pred.apply(prediction, item=0.5)
1 - np.sum(abs(new_pred - y_train)) / len(y_train)

0.9714285714285714

In [20]:
# Определим долю верных ответов на тестовой выборке
pred = my_prediction(params, X_test)
new_pred = pred.apply(prediction, item=0.5)
1 - np.sum(abs(new_pred - y_test)) / len(y_test)

0.9333333333333333

### RMSProp

In [21]:
# Задаём исходные данные
lr = 0.1
n_epochs = 50
params = []
params = np.random.normal(size=(5,))
gamma = 0.9
eps = 0.000001
grad_square = np.zeros(5)

In [22]:
for epoch in range(n_epochs):
    predictions_demo = my_prediction(params, X_train)
    grad_0 = np.sum((predictions_demo - y_train)) / len(predictions_demo)
    grad_1 = np.dot((predictions_demo - y_train), X_train) / len(predictions_demo)
    grad = np.hstack((np.array(grad_0), grad_1))
    grad_square = gamma * grad_square + (1 - gamma)  * grad ** 2
    params -= lr * grad / np.sqrt(grad_square + eps)
    predictions = my_prediction(params, X_train).apply(prediction, item=0.5)

In [23]:
# Определим долю верных ответов на тренировочной выборке
pred = my_prediction(params, X_train)
new_pred = pred.apply(prediction, item=0.5)
1 - np.sum(abs(new_pred - y_train)) / len(y_train)

0.7142857142857143

In [24]:
# Определим долю верных ответов на тестовой выборке
pred = my_prediction(params, X_test)
new_pred = pred.apply(prediction, item=0.5)
1 - np.sum(abs(new_pred - y_test)) / len(y_test)

0.6666666666666667